# OpenFedLLM 联邦学习训练 - Google Colab版本

本notebook用于在Google Colab上运行联邦学习训练，无需GPT API评估。

## 使用说明
1. 在Colab中运行时，确保选择GPU运行时（运行时 -> 更改运行时类型 -> GPU）
2. 按顺序执行所有cell
3. 训练完成后可以查看训练损失曲线

## 步骤1: 检查并安装依赖包（智能安装）

Colab已经预装了很多包，我们只安装缺失或版本不匹配的包。

In [ ]:
# 首先检查哪些包已经安装，哪些需要安装
import importlib
import subprocess
import sys

# 需要检查的包及其版本要求
required_packages = {
    'transformers': '4.31.0',
    'peft': '0.4.0',
    'trl': '0.7.2',
    'bitsandbytes': '0.40.2',
    'accelerate': '0.21.0',
    'datasets': '2.13.0',
    'torch': None,  # 检查是否安装，版本由Colab决定
    'tqdm': None,  # 通常已安装
    'numpy': None,  # 通常已安装
}

print("🔍 检查已安装的包...")
packages_to_install = []

for package, required_version in required_packages.items():
    try:
        module = importlib.import_module(package)
        installed_version = getattr(module, '__version__', 'unknown')
        
        if required_version:
            # 检查版本是否匹配
            if installed_version != required_version:
                print(f"  ⚠️  {package}: {installed_version} (需要 {required_version})")
                packages_to_install.append(f"{package}=={required_version}")
            else:
                print(f"  ✅ {package}: {installed_version} (已安装，版本正确)")
        else:
            print(f"  ✅ {package}: {installed_version} (已安装)")
    except ImportError:
        print(f"  ❌ {package}: 未安装")
        if required_version:
            packages_to_install.append(f"{package}=={required_version}")
        else:
            packages_to_install.append(package)

print("\n" + "="*60)
if packages_to_install:
    print(f"需要安装 {len(packages_to_install)} 个包:")
    for pkg in packages_to_install:
        print(f"  - {pkg}")
else:
    print("✅ 所有必需的包都已安装且版本正确！")
    print("可以跳过安装步骤")

In [ ]:
# 只安装缺失或版本不匹配的包
if packages_to_install:
    print(f"📦 正在安装 {len(packages_to_install)} 个包...")
    print("\n" + "="*60)
    
    # 批量安装（更高效）
    install_cmd = f"pip install -q {' '.join(packages_to_install)}"
    print(f"执行命令: {install_cmd}")
    
    result = subprocess.run(
        install_cmd.split(),
        capture_output=True,
        text=True
    )
    
    if result.returncode == 0:
        print("\n✅ 依赖包安装完成！")
    else:
        print(f"\n⚠️ 安装过程中可能有警告，但通常可以继续")
        if result.stderr:
            print(f"错误信息: {result.stderr[:500]}")
else:
    print("✅ 所有包都已安装，跳过安装步骤")

# 验证安装
print("\n🔍 验证安装结果...")
failed_packages = []
for package in required_packages.keys():
    try:
        importlib.import_module(package)
        print(f"  ✅ {package}")
    except ImportError:
        print(f"  ❌ {package} 导入失败")
        failed_packages.append(package)

if failed_packages:
    print(f"\n⚠️ 以下包导入失败: {failed_packages}")
    print("请手动安装: !pip install " + " ".join(failed_packages))
else:
    print("\n✅ 所有包验证通过！")

## 步骤2: 克隆代码库

In [ ]:
import os

# 克隆仓库（如果还没有）
repo_url = 'https://github.com/GuangLun2000/OpenFedLLM-Attack.git'
repo_name = 'OpenFedLLM-Attack'

if not os.path.exists(repo_name):
    print(f"正在克隆仓库: {repo_url}")
    !git clone --recursive --shallow-submodules {repo_url} {repo_name}
    print("✅ 代码库克隆完成！")
else:
    print("✅ 代码库已存在，跳过克隆")

# 进入项目目录
os.chdir(repo_name)
print(f"当前工作目录: {os.getcwd()}")

# 验证关键文件是否存在
required_files = ['main_sft.py', 'config.py', 'utils', 'federated_learning']
for file in required_files:
    if os.path.exists(file):
        print(f"  ✅ {file} 存在")
    else:
        print(f"  ❌ {file} 不存在！")

## 步骤3: 设置环境变量

In [ ]:
import os
import sys

# 设置PYTHONPATH
fingpt_dir = 'evaluation/FinGPT'
if os.path.exists(fingpt_dir):
    sys.path.insert(0, os.path.abspath(fingpt_dir))

# 将当前目录添加到Python路径
sys.path.insert(0, os.getcwd())

print("✅ 环境变量设置完成！")
print(f"Python路径: {sys.path[:3]}")

## 步骤4: 调试和验证环境

In [ ]:
# 验证所有必要的模块是否可以导入
import sys
import os

print("🔍 环境检查:")
print("\n1. 检查Python路径:")
print(f"   当前目录: {os.getcwd()}")
print(f"   Python路径包含: {os.getcwd() in sys.path}")

print("\n2. 检查关键文件:")
key_files = ['main_sft.py', 'config.py', 'utils/__init__.py', 'federated_learning/__init__.py']
for file in key_files:
    exists = os.path.exists(file)
    status = "✅" if exists else "❌"
    print(f"   {status} {file}")

print("\n3. 尝试导入关键模块:")
try:
    from utils import *
    print("   ✅ utils 模块导入成功")
except Exception as e:
    print(f"   ❌ utils 模块导入失败: {e}")

try:
    from federated_learning import *
    print("   ✅ federated_learning 模块导入成功")
except Exception as e:
    print(f"   ❌ federated_learning 模块导入失败: {e}")

try:
    from config import get_config
    print("   ✅ config 模块导入成功")
except Exception as e:
    print(f"   ❌ config 模块导入失败: {e}")

print("\n4. 检查依赖包:")
packages = ['transformers', 'peft', 'trl', 'bitsandbytes', 'accelerate', 'datasets', 'torch']
for pkg in packages:
    try:
        __import__(pkg)
        print(f"   ✅ {pkg}")
    except ImportError:
        print(f"   ❌ {pkg} 未安装")

print("\n✅ 环境检查完成！")

## 步骤5: 检查GPU和CUDA

In [ ]:
import torch

print(f"PyTorch版本: {torch.__version__}")
print(f"CUDA可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA版本: {torch.version.cuda}")
    print(f"GPU设备: {torch.cuda.get_device_name(0)}")
    print(f"GPU内存: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ 警告: 未检测到GPU，请确保在Colab中选择了GPU运行时！")
    print("   设置方法: 运行时 -> 更改运行时类型 -> 硬件加速器 -> GPU")

## 步骤6: 配置训练参数（适合Colab的小规模配置）

In [ ]:
# ========== 训练参数配置 ==========
# 这些参数已经优化为适合Colab的资源限制

training_config = {
    # 模型配置
    'model_name_or_path': 'meta-llama/Llama-2-7b-hf',  # 基础模型
    'use_peft': True,  # 使用LoRA（必须）
    'peft_lora_r': 16,  # LoRA rank（降低以节省显存）
    'peft_lora_alpha': 32,  # LoRA alpha
    'load_in_8bit': True,  # 8bit量化（必须，节省显存）
    
    # 数据集配置
    'dataset_name': 'vicgalle/alpaca-gpt4',  # 数据集
    'dataset_sample': 5000,  # 使用5000个样本（减少数据量）
    'template': 'alpaca',  # 模板
    
    # 联邦学习配置
    'fed_alg': 'fedavg',  # 联邦学习算法
    'num_clients': 5,  # 客户端数量（减少）
    'sample_clients': 2,  # 每轮采样客户端数
    'num_rounds': 10,  # 训练轮数（减少用于快速测试）
    
    # 训练配置
    'max_steps': 5,  # 每轮每个客户端的训练步数（减少）
    'batch_size': 4,  # 批次大小（降低以节省显存）
    'gradient_accumulation_steps': 4,  # 梯度累积（保持有效batch size）
    'learning_rate': 2e-5,  # 学习率
    'seq_length': 256,  # 序列长度（降低以节省显存）
    
    # 输出配置
    'output_dir': './output_colab',  # 输出目录
    'save_model_freq': 5,  # 每5轮保存一次模型
    'logging_steps': 10,  # 日志步数
    'seed': 2023,  # 随机种子
}

print("✅ 训练参数配置完成！")
print("\n配置摘要:")
for key, value in training_config.items():
    print(f"  {key}: {value}")

## 步骤7: 准备命令行参数

In [ ]:
# 构建命令行参数列表
args_list = ['main_sft.py']  # 脚本名称

# 添加所有参数
for key, value in training_config.items():
    if isinstance(value, bool):
        if value:
            args_list.append(f"--{key}")
    else:
        args_list.append(f"--{key}")
        args_list.append(str(value))

# 添加联邦学习特定参数（如果还没有）
if '--save_model_freq' not in args_list:
    args_list.extend([
        '--save_model_freq', str(training_config['save_model_freq']),
    ])

print("✅ 命令行参数准备完成！")
print(f"\n将执行的命令:")
print(f"python {' '.join(args_list)}")
print(f"\n参数数量: {len(args_list)}")

## 步骤8: 开始训练

In [ ]:
# 构建完整的命令字符串
cmd_args = args_list[1:]  # 移除脚本名称
cmd_str = ' '.join([str(arg) for arg in cmd_args])

print("🚀 开始联邦学习训练...")
print("⏰ 预计时间: 根据GPU性能，可能需要30分钟到2小时")
print(f"\n执行命令: python main_sft.py {cmd_str[:100]}...")
print("\n" + "="*60)
print("\n请在下面的cell中执行训练命令（使用!python）")
print("\n或者取消注释下面的代码自动执行:")

In [ ]:
# 执行训练命令
# 注意：在Colab中，使用!命令执行Python脚本是最可靠的方法

import sys
import subprocess
import os

# 确保在正确的目录
if not os.path.exists('main_sft.py'):
    print("❌ 错误: 找不到 main_sft.py，请确保在项目根目录")
    print(f"当前目录: {os.getcwd()}")
else:
    print("正在执行训练...")
    print(f"工作目录: {os.getcwd()}")
    print(f"Python路径: {sys.executable}")
    print("\n" + "="*60 + "\n")

    try:
        # 使用subprocess执行，捕获输出
        result = subprocess.run(
            [sys.executable, 'main_sft.py'] + cmd_args,
            check=False,
            capture_output=True,
            text=True,
            cwd=os.getcwd()
        )
        
        # 显示输出
        if result.stdout:
            print("标准输出:")
            print(result.stdout)
        
        if result.stderr:
            print("\n错误输出:")
            print(result.stderr)
        
        print("\n" + "="*60)
        if result.returncode == 0:
            print("✅ 训练完成！")
        else:
            print(f"❌ 训练过程中出现错误，返回码: {result.returncode}")
            print("\n💡 调试建议:")
            print("  1. 检查上面的错误输出")
            print("  2. 确保所有依赖已安装")
            print("  3. 检查GPU是否可用")
            print("  4. 尝试使用下面的直接执行方法")
            print("\n💡 或者手动执行下面的命令:")
            print(f"!python main_sft.py {cmd_str}")
    except Exception as e:
        print(f"❌ 执行出错: {e}")
        import traceback
        traceback.print_exc()
        print("\n💡 请手动执行下面的命令:")
        print(f"!python main_sft.py {cmd_str}")

## 步骤7-备选: 直接执行命令（如果上面的方法失败）

In [ ]:
# 方法2: 直接使用!命令执行（推荐，更可靠）
# 如果上面的subprocess方法失败，使用这个方法

print("💡 如果上面的方法失败，请使用下面的直接执行方法:")
print("\n" + "="*60)
print("执行命令（复制到新的cell中运行）:")
print("\n" + "="*60)
print(f"!python main_sft.py {cmd_str}")
print("\n" + "="*60)
print("\n或者直接运行下面的代码:")
print("\n（取消注释下面的代码）")

# 取消下面的注释来直接执行
# !python main_sft.py {cmd_str}

In [ ]:
# 方法2: 直接使用!命令执行（最可靠的方法）
# 在Colab中，直接使用!命令执行是最可靠的方法

import os

# 确保在正确的目录
if os.path.exists('main_sft.py'):
    print("🚀 准备执行训练命令...")
    print(f"工作目录: {os.getcwd()}")
    print("\n" + "="*60)
    print("请复制下面的命令到新的cell中执行:")
    print("\n" + "="*60)
    print(f"!python main_sft.py {cmd_str}")
    print("\n" + "="*60)
    print("\n或者直接在这个cell中执行（取消下面的注释）:")
    print("\n注意: 在Jupyter/Colab中，!命令需要在单独的cell中执行")
    
    # 下面的代码会在执行时显示命令，但实际执行需要用户手动创建新cell
    # 或者使用get_ipython().run_cell_magic()
    try:
        from IPython import get_ipython
        ipython = get_ipython()
        if ipython:
            print("\n✅ 检测到IPython环境，可以直接执行")
            print("取消下面代码的注释来执行:")
            # 取消注释下面这行来执行
            # ipython.run_cell_magic('bash', '', f'python main_sft.py {cmd_str}')
    except:
        pass
else:
    print("❌ 错误: 找不到 main_sft.py")
    print(f"当前目录: {os.getcwd()}")
    print("请确保在项目根目录")

## 步骤8: 可视化训练损失（无需API）

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

# 加载训练损失
loss_file = os.path.join(training_config['output_dir'], 'training_loss.npy')

if os.path.exists(loss_file):
    training_loss = np.load(loss_file)
    
    print(f"训练损失形状: {training_loss.shape}")
    print(f"客户端数量: {training_loss.shape[0]}")
    print(f"训练轮数: {training_loss.shape[1]}")
    
    # 绘制损失曲线
    plt.figure(figsize=(12, 6))
    
    # 绘制每个客户端的损失
    for client_id in range(training_loss.shape[0]):
        client_loss = training_loss[client_id]
        # 过滤掉-1（未参与训练的轮次）
        valid_rounds = np.where(client_loss >= 0)[0]
        if len(valid_rounds) > 0:
            plt.plot(valid_rounds + 1, client_loss[valid_rounds], 
                    alpha=0.6, label=f'Client {client_id}', linewidth=1.5)
    
    # 计算并绘制平均损失
    avg_loss = []
    for round_id in range(training_loss.shape[1]):
        round_losses = training_loss[:, round_id]
        valid_losses = round_losses[round_losses >= 0]
        if len(valid_losses) > 0:
            avg_loss.append(np.mean(valid_losses))
        else:
            avg_loss.append(np.nan)
    
    valid_avg_rounds = [i+1 for i, v in enumerate(avg_loss) if not np.isnan(v)]
    valid_avg_loss = [v for v in avg_loss if not np.isnan(v)]
    
    if len(valid_avg_rounds) > 0:
        plt.plot(valid_avg_rounds, valid_avg_loss, 
                'k-', linewidth=3, label='Average Loss', marker='o', markersize=6)
    
    plt.xlabel('Training Round', fontsize=12)
    plt.ylabel('Training Loss', fontsize=12)
    plt.title('Federated Learning Training Loss', fontsize=14, fontweight='bold')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # 打印统计信息
    print("\n📊 训练统计信息:")
    print(f"  初始平均损失: {valid_avg_loss[0]:.4f}")
    print(f"  最终平均损失: {valid_avg_loss[-1]:.4f}")
    print(f"  损失下降: {valid_avg_loss[0] - valid_avg_loss[-1]:.4f}")
    print(f"  损失下降百分比: {(valid_avg_loss[0] - valid_avg_loss[-1]) / valid_avg_loss[0] * 100:.2f}%")
    
else:
    print(f"❌ 未找到训练损失文件: {loss_file}")
    print("   请确保训练已完成并保存了损失文件")

## 步骤10: 检查输出文件

In [ ]:
import os
import json

output_dir = training_config['output_dir']

if os.path.exists(output_dir):
    print(f"✅ 输出目录存在: {output_dir}")
    print("\n📁 输出文件列表:")
    
    for item in os.listdir(output_dir):
        item_path = os.path.join(output_dir, item)
        if os.path.isdir(item_path):
            print(f"  📂 {item}/ (目录)")
        else:
            size = os.path.getsize(item_path)
            if size < 1024:
                size_str = f"{size} B"
            elif size < 1024 * 1024:
                size_str = f"{size / 1024:.2f} KB"
            else:
                size_str = f"{size / (1024 * 1024):.2f} MB"
            print(f"  📄 {item} ({size_str})")
    
    # 读取并显示配置
    config_file = os.path.join(output_dir, 'args.json')
    if os.path.exists(config_file):
        print("\n📋 训练配置:")
        with open(config_file, 'r') as f:
            config = json.load(f)
        print(json.dumps(config, indent=2, ensure_ascii=False))
else:
    print(f"❌ 输出目录不存在: {output_dir}")

## 💡 使用提示

### 如果遇到问题：

1. **显存不足 (OOM)**：
   - 进一步减小 `batch_size` (改为2或1)
   - 减小 `seq_length` (改为128)
   - 减小 `peft_lora_r` (改为8)
   - 减小 `dataset_sample` (改为2000)

2. **训练太慢**：
   - 减少 `num_rounds` (改为5)
   - 减少 `max_steps` (改为3)
   - 减少 `dataset_sample` (改为2000)

3. **模型下载失败**：
   - 确保已登录HuggingFace: `huggingface-cli login`
   - 或者使用本地已下载的模型路径

4. **想要更完整的训练**：
   - 增加 `num_rounds` (改为20-50)
   - 增加 `max_steps` (改为10)
   - 增加 `dataset_sample` (改为10000)
   - 增加 `num_clients` (改为10)

### 下一步：
- 训练完成后，模型保存在 `output_colab/checkpoint-*` 目录中
- 可以使用训练好的模型进行推理
- 如需评估，可以运行评估脚本（需要API key）